# **Notebook Setup**

In [2]:
dev = False
if dev:
  from google.colab import drive
  drive.mount('/content/drive')
else: 
  !git lfs install
  !git clone https://huggingface.co/migarbo1/metallica_lyric_gen

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'metallica_lyric_gen'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), 42.40 KiB | 6.06 MiB/s, done.


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.3 MB/s eta 0:00:00


# **Imports**

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

# **Model and Tokenizer**

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

# **Data Loading**

The path variable below defines there the root of the project is.

REPLACE IT WITH YOUR DESIRED ONE

In [6]:
root_path = 'drive/MyDrive/MetalliA/'# dataset/ models/

In the code below, the token `<endofverse>` represents the line break when finishing a verse in the song.

In [7]:
def parse_lyrics(root, song)-> str:
  '''
    Removes title and empty rows from song
    Returns lyrics without linebreaks
  '''
  if song.__contains__('.txt'):
    with open(os.path.join(root, song), "r") as file:
      lyrics = file.read()
      lyrics = [line+' <|endofverse|>' for line in lyrics.split('\n') if len(line) > 0]
      lyrics = ' '.join(lyrics)
  return lyrics

As the preprocess has already been done and we have stored the result as a csv file, we can load it. In case you want to provide another dataset of songs, the code is still there.

**song_name/album example for clarification**

root: drive/MyDrive/MetalliA/dataset/5-black_album

song: dont_tread_on_me.txt

In [8]:
if os.path.isfile('metallica_lyric_gen/metallica_song_dataset.csv'):
  df = pd.read_csv('metallica_lyric_gen/metallica_song_dataset.csv')
else:
  data_path = root_path + 'dataset/'

  df = pd.DataFrame(columns=['album', 'song_name', 'lyrics'])

  for root, dirs, files in os.walk(data_path):
    for song in files:
      #extract album name from path
      album = root.split('/')[-1]
      #remove album number from name
      album = album.split('-')[-1]

      #parse song name
      song_name = song.split('.')[0]
      song_name = ' '.join(song_name.split('_'))

      lyrics = parse_lyrics(root, song)

      row = {'album': album, 'song_name': song_name, 'lyrics': lyrics}
      df = df.append(row, ignore_index=True)


In [9]:
print("total of songs: {}".format(len(df.index)))

total of songs: 106


To check if the dataset has been created correctly, we inspect the first 10 rows.

In [10]:
df.head(10)

,Unnamed: 0,album,song_name,lyrics
0,0,ride_the_lightning,ride the lightning,"""Ride The Lightning"" <|endofverse|> Guilty as ..."
1,1,ride_the_lightning,escape,"""Escape"" <|endofverse|> Feel no pain, but my l..."
2,2,ride_the_lightning,trapped under ice,"""Trapped Under Ice"" <|endofverse|> I don't kno..."
3,3,ride_the_lightning,creeping death,"""Creeping Death"" <|endofverse|> Slaves <|endof..."
4,4,ride_the_lightning,fight fire with fire,"""Fight Fire With Fire"" <|endofverse|> Do unto ..."
5,5,ride_the_lightning,for whom the bell tolls,"""For Whom The Bell Tolls"" <|endofverse|> Make ..."
6,6,ride_the_lightning,fade to black,"""Fade To Black"" <|endofverse|> Life, it seems,..."
7,7,and_justice_for_all,shortest straw,"""The Shortest Straw"" <|endofverse|> Suspicion ..."
8,8,and_justice_for_all,the frayed ends of sanity,"""The Frayed Ends Of Sanity"" <|endofverse|> Nev..."
9,9,and_justice_for_all,eye of the beholder,"""Eye Of The Beholder"" <|endofverse|> Do you se..."


In order to have an idea of how long the songs are, we do a simple split and measure the mean and the mode.

In [11]:
lir = list(df['lyrics'])
song_length = [len(s.split(' ')) for s in lir]
print(song_length)
print('avg words: {}'.format(np.average(song_length)))
print('word median: {}'.format(np.median(song_length)))

[238, 270, 236, 287, 179, 191, 152, 384, 235, 413, 253, 358, 279, 206, 246, 312, 134, 229, 258, 266, 248, 207, 190, 441, 409, 354, 199, 138, 312, 217, 217, 244, 283, 213, 243, 276, 262, 347, 234, 305, 314, 280, 324, 260, 229, 170, 216, 193, 361, 285, 376, 210, 286, 346, 357, 327, 241, 213, 302, 368, 368, 161, 308, 499, 237, 272, 509, 183, 501, 275, 404, 599, 291, 310, 326, 322, 224, 481, 272, 348, 389, 371, 256, 379, 244, 258, 312, 244, 212, 263, 246, 260, 325, 335, 279, 433, 450, 254, 471, 264, 235, 249, 298, 419, 326, 332]
avg words: 293.5566037735849
word median: 275.5


We split the data into train and test set.

In [12]:
test_set = df.sample(n = 10)
train_set = df.loc[~df.index.isin(test_set.index)]
print('train set size: {}'.format(len(train_set.index)))
print('test set size: {}'.format(len(test_set.index)))

train set size: 96
test set size: 10


In [13]:
test_set = test_set.reset_index()
train_set = train_set.reset_index()

In the test set, we remove the last part of the songs in order to measure the accuracy of our trained model. Since the average number of words in the songs is less than 300, we're going to take a 10% of that out

In [14]:
test_set['true_lyrics'] = test_set['lyrics'].str.split().str[-30:].apply(' '.join)
test_set['lyrics'] = test_set['lyrics'].str.split().str[:-30].apply(' '.join)

In [15]:
test_set.head(10)

,index,Unnamed: 0,album,song_name,lyrics,true_lyrics
0,48,48,load,aint my bitch,"""Ain't My Bitch"" <|endofverse|> Outta my way <...",Taking me down <|endofverse|> Why you around <...
1,39,39,kill_em_all,seek and destroy,"""Seek & Destroy"" <|endofverse|> Alright <|endo...",Dying <|endofverse|> One thousand deaths <|end...
2,83,83,black_album,the unforgiven,"""The Unforgiven"" <|endofverse|> New blood join...",dub thee unforgiven. <|endofverse|> You labell...
3,20,20,master_of_puppets,lepper messiah,"""Leper Messiah"" <|endofverse|> Spineless from ...",will meet <|endofverse|> Make a contribution <...
4,75,75,death_magnetic,broken beat scarred,"""Broken, Beat & Scarred"" <|endofverse|> You ri...",in the hot sun's glory <|endofverse|> Show you...
5,11,11,and_justice_for_all,and justice for all,"""...And Justice For All"" <|endofverse|> Halls ...",Find it so grim <|endofverse|> So true <|endof...
6,17,17,72_seasons,if darkness had a son,"""If Darkness Had A Son"" <|endofverse|> Temptat...",<|endofverse|> If darkness had a son (Here I a...
7,47,47,load,poor twsited me,"""Poor Twisted Me"" <|endofverse|> Oh, poor twis...",soul again <|endofverse|> Good to feel my frie...
8,2,2,ride_the_lightning,trapped under ice,"""Trapped Under Ice"" <|endofverse|> I don't kno...",ice <|endofverse|> Freezing <|endofverse|> Can...
9,30,30,hardwired_to_self_destruct,man unkind,"""ManUNkind"" <|endofverse|> Chaos, <|endofverse...",Blind lead blind <|endofverse|> Quest to find ...


Create the dataset acording to GPT-2 expected format. 
Song max lenth is limited to 1024 tokens since GPT-2 performance gets worse from that length on.

We have to wrap it up into a class that inherits from torch Dataset so we can enumerate and loop over it.

In [16]:
class Lyrics(Dataset):
  def __init__(self, data, max_len = 1024):
    self.tokenizer = tokenizer
    self.lyrics = []

    for row in data['lyrics']:
      self.lyrics.append(
          torch.tensor(
            tokenizer.encode(f"<|[lyrics]|>{row[:max_len]}<|endoftext|>")
          )
      )
    
    self.count = len(self.lyrics)

  def __len__(self):
    return self.count

  def __getitem__(self, item):
    return self.lyrics[item]

In [17]:
tr_dataset = Lyrics(train_set)

# **Model Training**

First we define the hyper-parameters for our model

In [18]:
BATCH_SIZE = 24
EPOCHS = 100
LEARNING_RATE = 5e-5
WARM_STEPS = 200

models_path = root_path + 'models/'

The approach we've decided to fine-tune this model is based on gradient accumulation. Song by song we train the model and spread the loss and keep going. Once we reach as many songs as the batch size we've decided, that iteration we compute and spread the accumulated gradient. With 96 songs for training and batch size = 24, we make 4 gradient updates per epoch.

Additionaly, we save the best model in the `models/` folder marking the epoch in which has been achieved.

In [19]:
def train(
    dataset, output_prefix="MetalliA", save_model_on_epoch=True,
):
  gpt2.train()

  optimizer = AdamW(gpt2.parameters(), lr=LEARNING_RATE)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=WARM_STEPS, num_training_steps=-1
  )

  train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
  loss=0
  accumulating_batch_count = 0
  input_tensor = None
  min_loss = float("Inf")
  for epoch in range(EPOCHS):

    print(f"Training epoch {epoch}")
    
    for idx, entry in tqdm(enumerate(train_dataloader)):

      input_tensor = entry.to(device)
      outputs = gpt2(input_tensor, labels=input_tensor)
      loss = outputs[0]
      loss.backward()

      if (accumulating_batch_count % BATCH_SIZE) == 0:
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        gpt2.zero_grad()

      accumulating_batch_count += 1
      
    if save_model_on_epoch:
        if loss < min_loss:
          torch.save(
              gpt2.state_dict(),
              os.path.join(models_path, f"{output_prefix}-{epoch}.pt"),
          )
          min_loss = loss
    print(loss)
    print()
        
  return gpt2

In [20]:
device=torch.device("cuda")

Decide if you want to use the existing pre-trained model or if you want to play with the hyper-parameters and train your own model.

In [21]:
custom_train = False

In [22]:
if custom_train:
  gpt2 = gpt2.cuda()
  gpt2_finetune = train(tr_dataset)
else:
  gpt2_finetune = gpt2.cuda()
  gpt2_finetune.load_state_dict(torch.load('metallica_lyric_gen/MetalliA-best.pt'))
  gpt2_finetune.eval()

In [23]:
def generate(input, max_len = 30, top_p = 0.8, temp=1.0):
  gpt2_finetune.eval()

  device = torch.device("cuda")
  filter_value = -float("Inf")

  new_only = torch.tensor([]).unsqueeze(0).to(device)

  with torch.no_grad():
    generation = torch.tensor(tokenizer.encode(input)).unsqueeze(0).to(device)

    for i in tqdm(range(max_len)):
      outputs = gpt2_finetune(generation, labels = generation)
      loss, logits = outputs[:2]
      logits = logits[:, -1, :] / (temp if temp > 0 else 1.0)

      sorted_logits, sorted_index = torch.sort(logits, descending=True)
      cum_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim = -1)

      sorted_indices_to_remove = cum_probs > top_p
      sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
          ..., :-1
      ].clone()
      sorted_indices_to_remove[..., 0] = 0

      indices_to_remove = sorted_index[sorted_indices_to_remove]
      logits[:, indices_to_remove] = filter_value

      next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)

      if next_token in tokenizer.encode("<|endoftext|>"):
        break 

      new_only = torch.cat((new_only, next_token), dim=1)
      generation = torch.cat((generation, next_token), dim=1)
    
    output_list = list(generation.squeeze().cpu().numpy())
    output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 

    new_only_cpu = new_only.squeeze().cpu()

    if len(new_only_cpu.size()) > 0:
      only_gen_list = list(new_only_cpu.numpy())
      only_gen = f"{tokenizer.decode(only_gen_list)}<|endoftext|>"
    else:
      only_gen_list = []
      only_gen = f"<|endoftext|>"

  return output_text, only_gen

In [24]:
def text_generation(test_data):
  generated_lyrics = []
  full_lyrics = []
  for i in range(len(test_data)):
    full_song, gen_verses = generate(test_data['lyrics'][i])
    generated_lyrics.append(gen_verses)
    full_lyrics.append(full_song)
  return generated_lyrics, full_lyrics

In [25]:
generated_lyrics, full_lyrics = text_generation(test_set)

100%|██████████| 30/30 [00:01<00:00, 19.98it/s]


In [26]:
generated_lyrics

[' Ain’t mine <|endofverse|> Outta my way <|endofverse|> Outta my day <|end<|endoftext|>',
 ' Dying <|endofverse|> One thousand deaths <|endofverse|> Searching <|endofverse|> Seek and destroy<|endoftext|>',
 ' dub thee unforgiven. <|endofverse|> You branded me, <|endofverse|> You branded me, <|end<|endoftext|>',
 " will meet <|endofverse|> Make a contribution <|endofverse|> And you'll get the better seat <|endof<|endoftext|>",
 ', spitting your tears <|endofverse|> Show your scars <|endofverse|> Getting nowhere, nowhere soon <|endof<|endoftext|>',
 ' Find it so grim <|endofverse|> So real <|endofverse|> So real <|endofverse|> Just<|endoftext|>',
 '<|endoftext|>',
 ' soul again <|endofverse|> <|endoftext|>',
 ' ice <|endofverse|> Crystallized, as I lay here and rest <|endofverse|> Eyes of glass stare directly<|endoftext|>',
 ' Blind lead blind <|endofverse|> Quest to find <|endofverse|> Faith in man(un)kind <|end<|endoftext|>']

In [27]:
test_set['Generated_lyrics'] = generated_lyrics

In [28]:
test_set.head(10)

,index,Unnamed: 0,album,song_name,lyrics,true_lyrics,Generated_lyrics
0,48,48,load,aint my bitch,"""Ain't My Bitch"" <|endofverse|> Outta my way <...",Taking me down <|endofverse|> Why you around <...,Ain’t mine <|endofverse|> Outta my way <|endo...
1,39,39,kill_em_all,seek and destroy,"""Seek & Destroy"" <|endofverse|> Alright <|endo...",Dying <|endofverse|> One thousand deaths <|end...,Dying <|endofverse|> One thousand deaths <|en...
2,83,83,black_album,the unforgiven,"""The Unforgiven"" <|endofverse|> New blood join...",dub thee unforgiven. <|endofverse|> You labell...,dub thee unforgiven. <|endofverse|> You brand...
3,20,20,master_of_puppets,lepper messiah,"""Leper Messiah"" <|endofverse|> Spineless from ...",will meet <|endofverse|> Make a contribution <...,will meet <|endofverse|> Make a contribution ...
4,75,75,death_magnetic,broken beat scarred,"""Broken, Beat & Scarred"" <|endofverse|> You ri...",in the hot sun's glory <|endofverse|> Show you...,", spitting your tears <|endofverse|> Show your..."
5,11,11,and_justice_for_all,and justice for all,"""...And Justice For All"" <|endofverse|> Halls ...",Find it so grim <|endofverse|> So true <|endof...,Find it so grim <|endofverse|> So real <|endo...
6,17,17,72_seasons,if darkness had a son,"""If Darkness Had A Son"" <|endofverse|> Temptat...",<|endofverse|> If darkness had a son (Here I a...,<|endoftext|>
7,47,47,load,poor twsited me,"""Poor Twisted Me"" <|endofverse|> Oh, poor twis...",soul again <|endofverse|> Good to feel my frie...,soul again <|endofverse|> <|endoftext|>
8,2,2,ride_the_lightning,trapped under ice,"""Trapped Under Ice"" <|endofverse|> I don't kno...",ice <|endofverse|> Freezing <|endofverse|> Can...,"ice <|endofverse|> Crystallized, as I lay her..."
9,30,30,hardwired_to_self_destruct,man unkind,"""ManUNkind"" <|endofverse|> Chaos, <|endofverse...",Blind lead blind <|endofverse|> Quest to find ...,Blind lead blind <|endofverse|> Quest to find...


In [29]:
for i in range(len(test_set)):
  print("song name:{}".format(test_set['song_name'][i]))
  print("TRUE lyrics:\t\t{}".format(test_set['true_lyrics'][i]))
  print("Model generation:\t{}".format(test_set['Generated_lyrics'][i]))
  
  print()

song name:aint my bitch
TRUE lyrics:		Taking me down <|endofverse|> Why you around <|endofverse|> No fooling <|endofverse|> It ain’t my fall <|endofverse|> It ain’t my call <|endofverse|> It ain’t my bitch <|endofverse|> You ain’t mine <|endofverse|>
Model generation:	 Ain’t mine <|endofverse|> Outta my way <|endofverse|> Outta my day <|end<|endoftext|>

song name:seek and destroy
TRUE lyrics:		Dying <|endofverse|> One thousand deaths <|endofverse|> Searching <|endofverse|> Seek and destroy <|endofverse|> Searching <|endofverse|> Seek and destroy <|endofverse|> Searching <|endofverse|> Seek and destroy <|endofverse|> Searching <|endofverse|> Seek and destroy <|endofverse|>
Model generation:	 Dying <|endofverse|> One thousand deaths <|endofverse|> Searching <|endofverse|> Seek and destroy<|endoftext|>

song name:the unforgiven
TRUE lyrics:		dub thee unforgiven. <|endofverse|> You labelled me, <|endofverse|> I'll label you. <|endofverse|> So I dub thee unforgiven. <|endofverse|> Never fr

In [30]:
title = '"Design management: so boring" <|endofverse|>'
full_song, _ = generate(title, max_len=350, temp = 1.15)

100%|██████████| 350/350 [00:07<00:00, 46.11it/s]


In [31]:
print(full_song.replace('<|endofverse|>', '\n'))


"Design management: so boring" 
 I've got to have some help with it 
 I've got to make sure it looks good 
 I've got to make sure it has a clean feel 
 Make it wash faster 
 This suits me well when I run the risk 
 Just make sure it has the right dirt, right speed 
 Some people think this could just be me 
 Just do it right 
 Can't resist 
 Take the time to design 
 Draw out some of your chaos 
 Dare design on how boring the car is 
 How boring are they looking? 
 Look, I have time 
 Do I have the time? 
 Can't I just go ahead and do this right? 
 Wait a bit longer 
 We just got things fine, but how come they got this crap stuck in my head 
 Did I ask too much? 
 My mind's turned on design management 
 I've got to have some help with it 
 This suits me well when I run the risk 
 Just make sure it has the right dirt, right speed 
 Some people think this could just be me <|end<|endoftext|>
